In [1]:
!pip install sklearn
!pip install xgboost
!pip install keras

In [2]:
import ISee
import Profiler

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

import numpy as np

import Code_vin
import NullHypothesis

import re #REGEX sera utilizado para implementar as heuristicas de pre-processamento

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
def multisub(patterns, strns, text_pos):
    for strn in strns:
        for pattern, subst in patterns:
            strn[text_pos] = re.sub(pattern, subst, strn[text_pos])
    return strns

In [4]:
#A: bigrams
#B: all characters to lowercase
#C: accentuation removal
#D: special character treatment
#E: stop-words removal
#F: twitter user names removal
#G: twitter topics removal
#H: Reduction of laugh expressions to a common token
laugh_token = ' _laugh_ '
heuristics = {
    'A': [('not( [^ ]+)', 'not_\1')],
    'B': [('([A-Z]*)', lambda x: x.group(0).lower())],
    'C': [('á', 'a'), ('é', 'e'), ('í', 'i'), ('ó', 'o'), ('ú', 'u'), ('à', 'a'), ('ã', 'a'), ('ẽ', 'e'), ('ĩ', 'i'), ('õ', 'o'), ('ũ', 'u'),  ('â', 'a'), ('ê', 'e'), ('ñ', 'n')],
    'D': [(':\)', ''), (':\(', ''), (':\^\)', ''), (':p', ''), (':3', ''), (':c', ''), ('c:', ''), (':o', '')],
    'E': [(' the ', ' '), (' a ', ' '), (' e ', ' '), (' o ', ' ')],
    'F': [('@[^ ]+', ''), ('@[^ ]+ ', '')],
    'G': [('#[^ ]+', ''), ('#[^ ]+ ', '')],
    'H': [(' [ashu]{3,} ', laugh_token), (' [ah]{3,} ', laugh_token), (' [eh]{3,} ', laugh_token), (' k{4,} ', laugh_token), (' [rs]{3,} ', laugh_token), (' [ehu]{3,} ', laugh_token)],
}

In [5]:
#Configuracoes
heuristics_in_use = 'ABCDEFGH' #ABCDEFGH

max_features = 1000
embedding_dims = 50
maxlen = 50

kfold = 5

will_shuffle = True
shuffle_seed = 1234

n_est = 1000

In [6]:
prf = Profiler.Milestones()

[Profiler starting...]


In [7]:
print("Creating ensemble")
ensemble = ISee.Ensemble()
prf.add_milestone("Created ensemble object")

Creating ensemble


[Profiler] @ 18:02:31: Created ensemble object


In [8]:
print("Setting encoder & tokenizer")
ensemble.set_encoder(LabelEncoder())
ensemble.set_tokenizer(Tokenizer(max_features))
prf.add_milestone("Set encoder & tokenizer")

Setting encoder & tokenizer


[Profiler] @ 18:02:31: Set encoder & tokenizer


In [9]:
print("Reading file")
text_position = 4
class_position = 5
file_name = "brexit_blog_corpus.csv"
with open("datasets/" + file_name) as f:
    lines = [line.rstrip('\n').split('\t') for line in f][1:]
    print("Read {} lines".format(len(lines)))
    print("Applying heuristics " + heuristics_in_use + " (" + str(len(heuristics_in_use)) + ")")
    for h in heuristics_in_use:
        lines = multisub(heuristics[h], lines, text_position)
    line_no = 0
    for line in lines:
        line.append(line_no)
        line_no += 1
prf.add_milestone("Read file")

#print(str(lines[15]))
textos  = [str(a[text_position])  for a in lines]
classes = [str(b[class_position]) for b in lines]
n_classes = len(set(classes))
indexes = [c[-1] for c in lines]
prf.add_milestone("Aggregated data")

Reading file
Read 1682 lines
Applying heuristics ABCDEFGH (8)


[Profiler] @ 18:02:31: Read file
[Profiler] @ 18:02:31: Aggregated data


In [10]:
print("Setting data")
ensemble.set_datasets(textos, classes, indexes)
prf.add_milestone("Set datasets")

Setting data


[Profiler] @ 18:02:31: Set datasets


In [11]:
print("Fitting encoder & tokenizer")
ensemble.fit_encoder()
ensemble.fit_tokenizer()
prf.add_milestone("Fitted encoder & tokenizer")

Fitting encoder & tokenizer


[Profiler] @ 18:02:31: Fitted encoder & tokenizer


In [12]:
print("Splitting data")
ensemble.split_data(shuffle_seed=1337, purge_duplicates=False)
prf.add_milestone("Split data")

Splitting data


[Profiler] @ 18:02:31: Split data


In [13]:
print("Adding models")
##ensemble.add('Random', NullHypothesis.RandomClassifier())
#ensemble.add('XGB',  XGBClassifier(n_estimators=n_est, learning_rate=0.01, max_depth=6, subsample=0.65, colsample_bytree=0.25, gamma=5))
#ensemble.add('Forest', RandomForestClassifier(n_estimators=90))
#ensemble.add('Naive-B',  GaussianNB())
#ensemble.add('SVM_T', svm.SVC(gamma='scale'))
ensemble.add('RNC_1F_CF_1', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_1F_CF_2', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
#ensemble.add('RNC_1T_CF', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=max_features, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='binary_crossentropy'), uses_one_hot=True, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_1F_CT_1', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_1F_CT_2', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_1F_CT_3', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
ensemble.add('RNC_1F_CT_4', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=maxlen, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=False, uses_categorical=True, uses_argmax=True)
#ensemble.add('RNC_1T_CT', Code_vin.RNC_vin(max_features=max_features, embedding_dims=embedding_dims, maxlen=max_features, filters=100, kernel_size=3, hidden_dims=250, output_dims=n_classes, compile_loss='categorical_crossentropy'), uses_one_hot=True, uses_categorical=True, uses_argmax=True)
prf.add_milestone("Added models")

Adding models


[Profiler] @ 18:02:32: Added models


In [14]:
print("Training ensemble")
ensemble.train(max_features, maxlen, profiler=prf)
prf.add_milestone("Trained ensemble")

[Profiler] @ 18:02:32: Starting training
[Profiler] @ 18:02:32: Fold 1 - Training model RNC_1F_CT_4


Training ensemble
Training with the 1-st set


[Profiler] @ 18:02:46: Fold 1 - Trained model RNC_1F_CT_4
[Profiler] @ 18:02:46: Fold 1 - Training model RNC_1F_CT_2
[Profiler] @ 18:03:00: Fold 1 - Trained model RNC_1F_CT_2
[Profiler] @ 18:03:00: Fold 1 - Training model RNC_1F_CT_3
[Profiler] @ 18:03:13: Fold 1 - Trained model RNC_1F_CT_3
[Profiler] @ 18:03:13: Fold 1 - Training model RNC_1F_CF_1
[Profiler] @ 18:03:28: Fold 1 - Trained model RNC_1F_CF_1
[Profiler] @ 18:03:28: Fold 1 - Training model RNC_1F_CT_1
[Profiler] @ 18:03:42: Fold 1 - Trained model RNC_1F_CT_1
[Profiler] @ 18:03:42: Fold 1 - Training model RNC_1F_CF_2
[Profiler] @ 18:03:57: Fold 1 - Trained model RNC_1F_CF_2
[Profiler] @ 18:03:57: Fold 2 - Training model RNC_1F_CT_4


Training with the 2-nd set


[Profiler] @ 18:04:11: Fold 2 - Trained model RNC_1F_CT_4
[Profiler] @ 18:04:11: Fold 2 - Training model RNC_1F_CT_2
[Profiler] @ 18:04:25: Fold 2 - Trained model RNC_1F_CT_2
[Profiler] @ 18:04:25: Fold 2 - Training model RNC_1F_CT_3
[Profiler] @ 18:04:38: Fold 2 - Trained model RNC_1F_CT_3
[Profiler] @ 18:04:38: Fold 2 - Training model RNC_1F_CF_1
[Profiler] @ 18:04:52: Fold 2 - Trained model RNC_1F_CF_1
[Profiler] @ 18:04:52: Fold 2 - Training model RNC_1F_CT_1
[Profiler] @ 18:05:06: Fold 2 - Trained model RNC_1F_CT_1
[Profiler] @ 18:05:06: Fold 2 - Training model RNC_1F_CF_2
[Profiler] @ 18:05:20: Fold 2 - Trained model RNC_1F_CF_2
[Profiler] @ 18:05:20: Fold 3 - Training model RNC_1F_CT_4


...

[Profiler] @ 18:06:58: Fold 4 - Trained model RNC_1F_CT_4
[Profiler] @ 18:06:58: Fold 4 - Training model RNC_1F_CT_2
[Profiler] @ 18:07:12: Fold 4 - Trained model RNC_1F_CT_2
[Profiler] @ 18:07:12: Fold 4 - Training model RNC_1F_CT_3
[Profiler] @ 18:07:26: Fold 4 - Trained model RNC_1F_CT_3
[Profiler] @ 18:07:26: Fold 4 - Training model RNC_1F_CF_1
[Profiler] @ 18:07:43: Fold 4 - Trained model RNC_1F_CF_1
[Profiler] @ 18:07:43: Fold 4 - Training model RNC_1F_CT_1
[Profiler] @ 18:07:59: Fold 4 - Trained model RNC_1F_CT_1
[Profiler] @ 18:07:59: Fold 4 - Training model RNC_1F_CF_2
[Profiler] @ 18:08:14: Fold 4 - Trained model RNC_1F_CF_2
[Profiler] @ 18:08:15: Fold 5 - Training model RNC_1F_CT_4


Training with the 5-th set


[Profiler] @ 18:08:28: Fold 5 - Trained model RNC_1F_CT_4
[Profiler] @ 18:08:28: Fold 5 - Training model RNC_1F_CT_2
[Profiler] @ 18:08:42: Fold 5 - Trained model RNC_1F_CT_2
[Profiler] @ 18:08:42: Fold 5 - Training model RNC_1F_CT_3
[Profiler] @ 18:08:56: Fold 5 - Trained model RNC_1F_CT_3
[Profiler] @ 18:08:56: Fold 5 - Training model RNC_1F_CF_1
[Profiler] @ 18:09:09: Fold 5 - Trained model RNC_1F_CF_1
[Profiler] @ 18:09:09: Fold 5 - Training model RNC_1F_CT_1
[Profiler] @ 18:09:25: Fold 5 - Trained model RNC_1F_CT_1
[Profiler] @ 18:09:25: Fold 5 - Training model RNC_1F_CF_2


Finished training


[Profiler] @ 18:09:41: Fold 5 - Trained model RNC_1F_CF_2
[Profiler] @ 18:09:41: Trained ensemble


In [15]:
print("Testing ensemble")
res = ensemble.test(max_features, maxlen, profiler=prf)
prf.add_milestone("Tested ensemble")

[Profiler] @ 18:09:41: Starting testing
[Profiler] @ 18:09:41: Fold 1 - Testing model RNC_1F_CT_4


Testing ensemble
Testing with the 1-st set


[Profiler] @ 18:09:41: Fold 1 - Tested model RNC_1F_CT_4
[Profiler] @ 18:09:41: Fold 1 - Testing model RNC_1F_CT_2
[Profiler] @ 18:09:42: Fold 1 - Tested model RNC_1F_CT_2
[Profiler] @ 18:09:42: Fold 1 - Testing model RNC_1F_CT_3
[Profiler] @ 18:09:42: Fold 1 - Tested model RNC_1F_CT_3
[Profiler] @ 18:09:42: Fold 1 - Testing model RNC_1F_CF_1
[Profiler] @ 18:09:42: Fold 1 - Tested model RNC_1F_CF_1
[Profiler] @ 18:09:42: Fold 1 - Testing model RNC_1F_CT_1
[Profiler] @ 18:09:42: Fold 1 - Tested model RNC_1F_CT_1
[Profiler] @ 18:09:42: Fold 1 - Testing model RNC_1F_CF_2
[Profiler] @ 18:09:42: Fold 1 - Tested model RNC_1F_CF_2


Testing with the 2-nd set


[Profiler] @ 18:09:42: Fold 2 - Testing model RNC_1F_CT_4
[Profiler] @ 18:09:43: Fold 2 - Tested model RNC_1F_CT_4
[Profiler] @ 18:09:43: Fold 2 - Testing model RNC_1F_CT_2
[Profiler] @ 18:09:43: Fold 2 - Tested model RNC_1F_CT_2
[Profiler] @ 18:09:43: Fold 2 - Testing model RNC_1F_CT_3
[Profiler] @ 18:09:43: Fold 2 - Tested model RNC_1F_CT_3
[Profiler] @ 18:09:43: Fold 2 - Testing model RNC_1F_CF_1
[Profiler] @ 18:09:43: Fold 2 - Tested model RNC_1F_CF_1
[Profiler] @ 18:09:43: Fold 2 - Testing model RNC_1F_CT_1
[Profiler] @ 18:09:43: Fold 2 - Tested model RNC_1F_CT_1
[Profiler] @ 18:09:43: Fold 2 - Testing model RNC_1F_CF_2
[Profiler] @ 18:09:43: Fold 2 - Tested model RNC_1F_CF_2


...

[Profiler] @ 18:09:43: Fold 3 - Testing model RNC_1F_CT_4
[Profiler] @ 18:09:43: Fold 3 - Tested model RNC_1F_CT_4
[Profiler] @ 18:09:43: Fold 3 - Testing model RNC_1F_CT_2
[Profiler] @ 18:09:43: Fold 3 - Tested model RNC_1F_CT_2
[Profiler] @ 18:09:43: Fold 3 - Testing model RNC_1F_CT_3
[Profiler] @ 18:09:43: Fold 3 - Tested model RNC_1F_CT_3
[Profiler] @ 18:09:43: Fold 3 - Testing model RNC_1F_CF_1
[Profiler] @ 18:09:43: Fold 3 - Tested model RNC_1F_CF_1
[Profiler] @ 18:09:43: Fold 3 - Testing model RNC_1F_CT_1
[Profiler] @ 18:09:43: Fold 3 - Tested model RNC_1F_CT_1
[Profiler] @ 18:09:43: Fold 3 - Testing model RNC_1F_CF_2
[Profiler] @ 18:09:43: Fold 3 - Tested model RNC_1F_CF_2
[Profiler] @ 18:09:43: Fold 4 - Testing model RNC_1F_CT_4
[Profiler] @ 18:09:43: Fold 4 - Tested model RNC_1F_CT_4
[Profiler] @ 18:09:43: Fold 4 - Testing model RNC_1F_CT_2


Testing with the 4-th set


[Profiler] @ 18:09:43: Fold 4 - Tested model RNC_1F_CT_2
[Profiler] @ 18:09:43: Fold 4 - Testing model RNC_1F_CT_3
[Profiler] @ 18:09:43: Fold 4 - Tested model RNC_1F_CT_3
[Profiler] @ 18:09:43: Fold 4 - Testing model RNC_1F_CF_1
[Profiler] @ 18:09:43: Fold 4 - Tested model RNC_1F_CF_1
[Profiler] @ 18:09:43: Fold 4 - Testing model RNC_1F_CT_1
[Profiler] @ 18:09:44: Fold 4 - Tested model RNC_1F_CT_1
[Profiler] @ 18:09:44: Fold 4 - Testing model RNC_1F_CF_2
[Profiler] @ 18:09:44: Fold 4 - Tested model RNC_1F_CF_2
[Profiler] @ 18:09:44: Fold 5 - Testing model RNC_1F_CT_4
[Profiler] @ 18:09:44: Fold 5 - Tested model RNC_1F_CT_4
[Profiler] @ 18:09:44: Fold 5 - Testing model RNC_1F_CT_2
[Profiler] @ 18:09:44: Fold 5 - Tested model RNC_1F_CT_2
[Profiler] @ 18:09:44: Fold 5 - Testing model RNC_1F_CT_3
[Profiler] @ 18:09:44: Fold 5 - Tested model RNC_1F_CT_3
[Profiler] @ 18:09:44: Fold 5 - Testing model RNC_1F_CF_1


Testing with the 5-th set
Finished testing


[Profiler] @ 18:09:44: Fold 5 - Tested model RNC_1F_CF_1
[Profiler] @ 18:09:44: Fold 5 - Testing model RNC_1F_CT_1
[Profiler] @ 18:09:44: Fold 5 - Tested model RNC_1F_CT_1
[Profiler] @ 18:09:44: Fold 5 - Testing model RNC_1F_CF_2
[Profiler] @ 18:09:44: Fold 5 - Tested model RNC_1F_CF_2
[Profiler] @ 18:09:44: Tested ensemble


In [16]:
ensemble.evaluate(verbose=True)
prf.add_milestone("Finished evaluating")

Evaluating 1-th fold
	     -  Correct - RNC_1F_CT_4 - RNC_1F_CT_2 - RNC_1F_CT_3 - RNC_1F_CF_1 - RNC_1F_CT_1 - RNC_1F_CF_2 -   Major.- Phrase
	   0 -        0 -       0* -       7  -       0* -       2  -       0* -       0* -       0* i know what you mean.
	   1 -        0 -       2  -       0* -       0* -       2  -       2  -       2  -       2  i don't disagree that yes campaign made mistakes, but they must be contextualised within truly immense opposition they had.
	   2 -        0 -       0* -       7  -       0* -       2  -       2  -       1  -       0* of course, this is far from dominant narrative we hear in media.
	   3 -        0 -       2  -       2  -       2  -       2  -       2  -       2  -       2  in principle i agree with what he is striving for, but in practice it is not_ simple.
	   4 -        0 -       1  -       7  -       0* -       2  -       2  -       1  -       1  however, i still remain flummoxed as to why expats get to vote, it is no longer their conce

	 282 -        7 -       9  -       7* -       9  -       4  -       4  -       1  -       9  i just want to get hell out.
	 283 -        7 -       4  -       7* -       7* -       4  -       6  -       1  -       4  fuck infirm, old, young and unemployed.
	 284 -        7 -       4  -       7* -       7* -       4  -       6  -       1  -       4  thanks for your response.
	 285 -        7 -       4  -       7* -       9  -       2  -       2  -       1  -       2  he's been around long time, and in all that time i've never thought of him as anything other than very-far-left, israel-loathing, hamas-hizbollah-friendly, pro-sinn fein labour party fringe player, part of 'awkward squad', utterly contemptible and unimportant.
	 286 -        7 -       4  -       7* -       7* -       4  -       3  -       1  -       4  i fear your british balls are shrinking.
	 287 -        7 -       4  -       7* -       7* -       3  -       3  -       1  -       7* an offender would be described simply a

[Profiler] @ 18:09:44: Finished evaluating


In [17]:
print("Profiler data:")
prf.exhibit(max_d=10)

Profiler data:
0==========1=====2=====3=====4=5======6====7======8====9=========10=====11=====12==13========14=====15=====16=17=====18======19=====20======21=======22=====23=====24=====25=====26=====27=====28=====29=====30=====31=====32=====33=====34==35=====36========37=====38=====39=====40=====41=====42=43=====44========45=======46=====47=====48=====49=50=========51=52=====53=========54======55=====56===57=======58====59==60====61========62=====63======64=====65=====66===67=======68=====69==70=====71========72=====73=====74=====75=====76=====77=====78=====79=====80=81=========82=====83==84=======85=====86======87====88=====89====90===91=====92======93=====94=========95=====96=97=======98====99========100====101======102=====103=====104=====105===106=====107=======108===109======110===111=====112========113=====114=====115===116=====117==118=========119=120========121====122=====123==124=========125=====126==127====128=========129=====130=====131===132=====133
  0: Start - 22/04/2019 